In [1]:
import sys
from pathlib import Path
project_root = Path().resolve().parent
src_path = project_root / 'src'
config_path = project_root / 'config.py'
if config_path.exists():
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
    print('✅ config.py aan het pad toegevoegd')
if src_path.exists():
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    print('✅ src aan het pad toegevoegd')

✅ config.py aan het pad toegevoegd
✅ src aan het pad toegevoegd


In [2]:
# 🔧 Test config.py import
from config import EXPORT_FOLDER, CSV_FOLDER, SHAPEFILE_FOLDER

print('📁 EXPORT_FOLDER:', EXPORT_FOLDER)
print('📁 CSV_FOLDER:', CSV_FOLDER)
print('📁 SHAPEFILE_FOLDER:', SHAPEFILE_FOLDER)


📁 EXPORT_FOLDER: notebooks/exports
📁 CSV_FOLDER: data/csv
📁 SHAPEFILE_FOLDER: data/shapefiles


In [3]:
import config
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import csv
import os

from data_service import get_metric_mapping, get_available_years, get_municipality_heatmap_data, get_province_heatmap_data, get_prepared_covid_dataset

EXPORT_DIR = Path.cwd() / "exports"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)
LOG_FILE = EXPORT_DIR / "export_log.csv"


In [4]:
def plot_heatmap_scaled(gdf, column, title="", vmin=None, vmax=None, unit="", export=False, filename=None):
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(column=column, ax=ax, legend=True, edgecolor='white', linewidth=0.2,
             vmin=vmin, vmax=vmax)
    ax.set_title(title, fontsize=14)
    ax.axis("off")
    ax.annotate('N', xy=(0.86, 0.12), xytext=(0.86, 0.05),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=12, xycoords=ax.transAxes)
    cbar = fig.axes[-1]
    cbar.set_ylabel(unit, rotation=90, labelpad=15)
    if export and filename:
        filepath = EXPORT_DIR / filename
        plt.savefig(filepath, dpi=300, bbox_inches="tight")
        print(f"✅ PNG exported to: {filepath}")
        log_export("Export heatmap as PNG", filename)
    plt.show()

def log_export(export_type, filename):
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with LOG_FILE.open("a", newline="") as f:
        csv.writer(f).writerow([now, export_type, filename])


In [5]:
unit_label = "Cases/100K"
df = get_prepared_covid_dataset()

year_options = get_available_years(df)
metric_mapping = get_metric_mapping()
metric_options = list(metric_mapping.keys())

year_dropdown = widgets.Dropdown(options=year_options, value=year_options[-1], description="Year:",
                                 layout=widgets.Layout(width="320px"))
metric_dropdown = widgets.Dropdown(options=metric_options, value=metric_options[0], description="Metric:",
                                   layout=widgets.Layout(width="320px"))
region_dropdown = widgets.Dropdown(options=["Provinces", "Municipalities"], value="Provinces", description="Level:",
                                   layout=widgets.Layout(width="320px"))

# Gebruik max van een relevante kolom om schaal te bepalen
sample_df = get_municipality_heatmap_data(year_options[-1])
sample_max = int(np.ceil(sample_df[metric_mapping[metric_options[0]]].max() / 100.0)) * 100
scale_options = [int(sample_max * r) for r in [0.2, 0.4, 0.6, 0.8, 1.0]]

scale_dropdown = widgets.Dropdown(
    options=[f"{val} {unit_label}" for val in scale_options],
    value=f"{scale_options[-1]} {unit_label}",
    description="Scale max:",
    layout=widgets.Layout(width="320px")
)

export_dropdown = widgets.Dropdown(
    options=["Select export...", "Export as CSV", "Export as Markdown", "Export as PDF", "Export heatmap as PNG"],
    description="Export:",
    layout=widgets.Layout(width="320px")
)
export_button = widgets.Button(description="Go", button_style="success", icon="download",
                               layout=widgets.Layout(width="80px"))

title_widget = widgets.HTML("<h3 style='text-align:left;'>Heatmap – Regional COVID Data</h3>")
plot_output = widgets.Output()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
left_column = widgets.VBox([
    year_dropdown,
    metric_dropdown,
    region_dropdown,
    scale_dropdown,
    widgets.HBox([export_dropdown, export_button])
], layout=widgets.Layout(width="320px", justify_content='flex-start'))

ui_layout = widgets.HBox([
    left_column,
    plot_output
], layout=widgets.Layout(justify_content='flex-start', align_items='flex-start', width='100%', padding='10px'))

display(widgets.VBox([title_widget, ui_layout], layout=widgets.Layout(width='100%', overflow='hidden')))


In [ ]:
def update_scale_options():
    year = year_dropdown.value
    metric_key = metric_dropdown.value
    metric_col = metric_mapping[metric_key]
    region = region_dropdown.value
    gdf = get_province_heatmap_data(year) if region == "Provinces" else get_municipality_heatmap_data(year)
    data_max = gdf[metric_col].max()
    if pd.isnull(data_max):
        data_max = 1  # fallback
    base = int(np.ceil(data_max / 100.0)) * 100
    values = [int(base * r) for r in [0.2, 0.4, 0.6, 0.8, 1.0]]
    scale_dropdown.options = [f"{v} {unit_label}" for v in values]
    scale_dropdown.value = f"{values[-1]} {unit_label}"


In [ ]:
def get_scale_value(scale_dropdown):
    return int(scale_dropdown.value.split()[0])


In [ ]:
def update_plot(change=None):
    update_scale_options()
    plot_output.clear_output()
    with plot_output:
        year = year_dropdown.value
        metric_key = metric_dropdown.value
        metric = metric_mapping[metric_key]
        region = region_dropdown.value
        gdf = get_province_heatmap_data(year) if region == "Provinces" else get_municipality_heatmap_data(year)
        vmax = get_scale_value(scale_dropdown)
        plot_heatmap_scaled(gdf, metric, f"Heatmap – {region}", vmin=0, vmax=vmax, unit=unit_label)

region_dropdown.observe(update_plot, names='value')
year_dropdown.observe(update_plot, names='value')
metric_dropdown.observe(update_plot, names='value')
scale_dropdown.observe(update_plot, names='value')
update_plot()
